<a href="https://colab.research.google.com/github/ollihansen90/VectorQuantisierung_Futureskills/blob/main/VecQuant_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitel 6: Beispielanwendung Farbreduktion
In diesem Notebook soll als Beispielanwendung die Farbreduktion aus den Videos aufgegriffen werden.

In [ ]:
# TODO: Auf dem Jupyter-Hub werden die utils.py sowie die Bilder bereits lokal gespeichert und muss nicht mit wget von Github gezogen werden.
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/VectorQuantisierung_Futureskills/main/utils.py
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/VectorQuantisierung_Futureskills/main/Mohnfeld.jpg
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/VectorQuantisierung_Futureskills/main/Rapsfeld.jpg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets
from IPython.display import display
from skimage import io
from utils import load_img, imshow, rgb2hex, hex2rgb, pointwise_sq, plot_img_w_points, kmeans_update, build_vq_img, zugehoerigkeit
from tqdm.auto import trange

## Wahl eines Bildes
Im folgenden Codeblock kann ein Bild ausgewählt werden. Die Bilder werden hinterher ein wenig verkleinert, sodass die längste Seite maximal 1000 Pixel lang ist. Der Grund hierfür ist, dass die Algorithmen so schneller laufen, ohne dass die Bildqualität groß darunter leidet.

TODO: Eigene Bilder? Entweder als Link oder durch Hochladen (das müsste im Jupyter-Hub getestet werden). Außerdem funktioniert das hübsche Auswählen wie hier nur bei Colab.

In [ ]:
Bild = 'Mohnfeld' # @param ["Mohnfeld", "Rapsfeld"]

img = load_img(Bild)

imshow(img)

colorlist = np.reshape(img, (-1, 3))
print(f"Bild geladen, {len(np.unique(colorlist, axis=0))} unterschiedliche Farben.")

## Farben wählen
In diesem Abschnitt können die Codebookvektoren "per Hand" gewählt werden. Hierfür muss der folgende Codeblock ausgeführt werden, woraufhin zunächst das Bild angezeigt wird und die Farben über einen Colorpicker ausgesucht werden soll.

*Hinweis:* Dieser Codeblock braucht nur ein Mal ausgeführt zu werden. Sämtliche Farben, wie sie ausgewählt werden, sind anschließend im Arbeitsspeicher, bis die Session beendet wird.

In [ ]:
#colors = colorlist[np.random.randint(0,len(colorlist), (8,))]
colors = np.random.rand(8,3)
#print(colors)
clist = [ipywidgets.ColorPicker(value=rgb2hex(c-0.001), layout=ipywidgets.Layout(width='100px')) for c in colors]

imshow(img)
box = ipywidgets.VBox([ipywidgets.HBox(clist[::2]), ipywidgets.HBox(clist[1::2])])
display(box)

## Rekonstruktion "per Hand"
Die soeben ausgewählten Farben können nun als Quantisierung im Farbraum des Bildes genutzt werden. Der folgende Codeblock führt die Quantisierung automatisch durch und plottet anschließend sowohl das Bild, als auch die zugehörige Punktewolke.

### Aufgabe
Suchen Sie eine gute Kombination aus Farben, sodass das Bild möglichst gut rekonstruiert wird! Sowohl für Mohn- als auch für Rapsfeld ist es möglich, "mit bloßem Auge" Farben zu finden, sodass der Fehler kleiner als 0.2 ist.

*Achtung:* Der obere Codeblock muss nicht erneut ausgeführt werden! Werden die Farben geändert, liegen sie bereits im Arbeitsspeicher vor. Anschließend muss der untere Codeblock werden.

In [ ]:
codebook = np.array([hex2rgb(c.value) for c in clist])
dists = pointwise_sq(colorlist, codebook)
naechster = np.argmin(dists, axis=-1)
img_neu_list = codebook[naechster]
img_neu = np.reshape(img_neu_list, img.shape)

print("Bild (links) mit Farbraum als Punktewolke (rechts)")
plot_img_w_points(img)

print("Quantisiertes Bild (links) mit zugehöriger Punktewolke (rechts)")
print("Der Fehler beträgt", np.mean(np.sqrt(np.sum((img_neu_list-colorlist)**2, axis=-1))))
plot_img_w_points(img_neu, [np.unique(colorlist, axis=0)[::20], codebook[np.argmin(pointwise_sq(np.unique(colorlist, axis=0), codebook), axis=-1)[::20]]])

## Rekonstruktion durch kMeans
Abschließend möchten wir uns ansehen, was der kMeans-Algorithmus erreichen kann. Hierfür werden zunächst die Codebookvektoren zufällig initialisiert und danach vom Algorithmus angepasst. Damit der Algorithmus nicht unendlich lange weiterläuft, wird er abgebrochen, sobald sich die Codebookvektoren nur noch wenig verändern.

### Aufgabe
Testen Sie den Algorithmus auch mit anderen Werten für $k$! Ab wann ist kaum noch ein Unterschied zwischen echtem und rekonstruierten Bild zu sehen?

In [ ]:
k = 8

max_epochs = 10
codebook = np.random.rand(k, 3)

img_neu = build_vq_img(colorlist, codebook, shape=img.shape)
_, error = zugehoerigkeit(codebook, colorlist)
print(f"Bild mit {k} unterschiedlichen Farben hat bei Initialisierung einen Fehler von {error}")
plot_img_w_points(img_neu, [np.unique(colorlist, axis=0)[::20], codebook[np.argmin(pointwise_sq(np.unique(colorlist, axis=0), codebook), axis=-1)[::20]]])

errorlist = []
for epoch in trange(max_epochs):
    cb_alt = codebook.copy()
    codebook, error = kmeans_update(codebook, colorlist)
    errorlist.append(error)
    if np.max(np.sum((cb_alt-codebook)**2, axis=-1))<1e-3:
        break


plt.figure()
plt.plot(errorlist)
plt.title("Fehler")
plt.show()

img_neu = build_vq_img(colorlist, codebook, shape=img.shape)
print(f"Bild mit {k} unterschiedlichen Farben hat am Ende einen Fehler von {errorlist[-1]}")
plot_img_w_points(img_neu, [np.unique(colorlist, axis=0)[::20], codebook[np.argmin(pointwise_sq(np.unique(colorlist, axis=0), codebook), axis=-1)[::20]]])